In [3]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3
import pickle
import re

from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from lxml import etree
from collections import defaultdict
from pprint import pprint
from config.assistant import *
from config.site_config import *
from tqdm import tqdm

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

request_headers = {
    'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36'),
}

In [2]:
search_words = ["컴퓨터비전", 'COMPUTER VISION',
                "영상처리", "IMAGE PROCESSING",
                "딥러닝", "DEEP LEARNING", 
                "머신러닝", "MACHINE LEARNING",
                "이미지인식", "IMAGE RECOGNITION",
                "이미지분석", "IMAGE ANALYSIS",
                "VISION AI",
                "OPENCV"
               ]

search_words = ["이미지인식"]
page_view_items = 100
company_cd='0,1,2,3,4,5,6,7'
base_link = 'https://www.saramin.co.kr'
job_group = {}


In [2]:
len(job_group)

NameError: name 'job_group' is not defined

In [12]:
# 머신러닝: 1450
# MACHINE LEARNING: 1445
# 딥러닝: 1158
# DEEP LEARNING: 1158
# 컴퓨터비전: 1120
# 영상처리: 1037
# COMPUTER VISION: 964
# 이미지분석: 714
# VISION AI: 657
# IMAGE PROCESSING: 295
# OPENCV: 232
# 이미지인식: 222
# IMAGE ANALYSIS: 21
# IMAGE RECOGNITION: 12

# 9365
# job_group 2259

In [13]:
# JSON 파일 쓰기
file_path = "config/job_group.json"

tf = open(file_path, 'w')
json.dump(job_group, tf)
tf.close()

In [2]:
# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)

In [3]:
print(len(job_group))

24


In [4]:
copy_group = {}

In [5]:
input_status = 5
if input_status == 1:      status = 'all'
elif input_status == 2:    status = 'wait'
elif input_status == 3:    status = 'save'
elif input_status == 4:    status = 'hold'
elif input_status == 5:    status = 'close'
        
# ALL   : 1
# WAIT  : 2
# SAVE  : 3
# HOLD  : 4
# CLOSE : 5

def filter_status_func():
    try:
        copy_group[companys].append(job_group[companys][idx])
    except:
        copy_group[companys] = []
        copy_group[companys].append(job_group[companys][idx])
    
for companys in job_group:
    for idx, job_list in enumerate(job_group[companys]):
        if job_group[companys][idx]['status'] == status:    # ALL
            test_func()
        elif job_group[companys][idx]['status'] == status:    # WAIT
            test_func()
        elif job_group[companys][idx]['status'] == status:    # SAVE
            test_func()
        elif job_group[companys][idx]['status'] == status:    # HOLD
            test_func()
        elif job_group[companys][idx]['status'] == status:    # CLOSE
            test_func()   

In [6]:
pprint(copy_group)

{'스텔란티스코리아': [{'company': '스텔란티스코리아',
               'company_link': 'https://www.saramin.co.kr/zf_user/company-info/view?csn=QWZyVlFHKy9RQkNRQmhSVXRDWnJ5UT09',
               'input_date': '2022-08-28',
               'status': 'close',
               'title': '[수입차 Jeep/Peugeot] Product Planning (대리-과장급)',
               'title_idx': 1,
               'title_link': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=search&rec_idx=43852302&location=ts&searchword=IMAGE+ANALYSIS&searchType=search&paid_fl=n&search_uuid=1fea4ee1-80dd-49cb-a985-e7bbcf9629ff'}],
 '제이엘케이': [{'company': '제이엘케이',
            'company_link': 'https://www.saramin.co.kr/zf_user/company-info/view?csn=dGJHczlpN0NvYWNkaEhUbGRXZXdKUT09',
            'input_date': '2022-08-28',
            'status': 'close',
            'title': '[JLK] 인공지능(AI)·빅데이터연구원 채용(병역특례가능)',
            'title_idx': 0,
            'title_link': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=search&rec_idx=43622025&locati

In [3]:
# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)
print(len(job_group))

5935


In [4]:
job_company = '(주)샤플(SHAPL)'
job_company

'(주)샤플(SHAPL)'

In [5]:
# 회사명 - 특정 단어 제거
for remove_word in remove_words:
    if remove_word in job_company:
        job_company = job_company.replace(remove_word, '')

In [6]:
job_company

'샤플(SHAPL)'

In [9]:
company_re = re.search('\(.*\)', job_company)
company_re


<re.Match object; span=(2, 9), match='(SHAPL)'>

In [10]:
str_filtering = job_company[company_re.span()[0]:company_re.span()[1]]
str_filtering

'(SHAPL)'

In [12]:
job_company = job_company.replace(str_filtering, '')
job_company

'샤플'

In [ ]:
if company_re:
    str_filtering = job_company[company_re.span()[0]:company_re.span()[1]]
    job_company = job_company.replace(str_filtering, '')

In [13]:
def filtering_company_name(job_company):

    # 회사명 - 특정 단어 제거
    for remove_word in remove_words:
        if remove_word in job_company:
            job_company = job_company.replace(remove_word, '')
            
    # 회사명 - ()괄호 안의 모든 단어 제거
    company_re = re.search('\(.*\)', job_company)
    if company_re:
        str_filtering = job_company[company_re.span()[0]:company_re.span()[1]]
        job_company = job_company.replace(str_filtering, '')
        

    


    return 1, job_company

In [15]:
filtering_company_name(job_company)

(1, '샤플')

In [16]:
for i in job_group['써머스플랫폼']:
    print(i['title'])

[에누리가격비교] 써머스플랫폼 검색개발 경력직 모집
[에누리가격비교] 데이터엔지니어 / 데이터사이언티스트 채용
[에누리가격비교] 데이터엔지니어 / 데이터사이언티스트 채용 Spring boot, JS, GA, Log, 수집, 분석, JAVA,ML, R, python, 추천, NLP
데이터 사이언티스트  (ML, R, python,  추천, NLP)
[에누리가격비교] 2022 써머스플랫폼 하반기 부문별 신입/경력 모집


In [4]:
job_company = "(재)대한불교진흥원"

In [5]:
# 회사명 - 특정 단어 제거
for remove_word in remove_words:
    if remove_word in job_company:
        job_company = job_company.replace(remove_word, '')

In [6]:
remove_words

[' ', '\n', '주식회사', '(주)', '(재)㈜', '㈔', '재)', '주)']

In [6]:
job_company

'대한불교진흥원'

In [2]:


# 회사명 - ()괄호 안의 모든 단어 제거
company_re = re.search('\(.*\)', job_company)
if company_re:
    str_filtering = job_company[company_re.span()[0]:company_re.span()[1]]
    job_company = job_company.replace(str_filtering, '')

# 회사명 - 완전히 일치하는 회사명 리스트 필터링
if job_company in filter_list_company_name:
    return 0, "none"

# 회사명 - 특정 단어가 포함되어 있는 회사 필터링
for remove_word in filter_list_company_name_part:
    if remove_word in job_company:
        return 0, "none"

return 1, job_company